In [326]:
import pandas as pd
# from ydata_profiling import ProfileReport

In [327]:
df = pd.read_csv('csv/uspevaemost.csv', encoding='cp1251')


In [328]:
df = df[df['Статус'] == 'Отчисленный']
df = df[df['Квалификация'] == 'Бакалавр']
df = df[df['Форма обучения'] == 'Очная']

In [329]:
df['sem_start'] = df['Учебный год'].str[:4]
df['sem_end'] = df['Учебный год'].str[5:]

df_years = df.groupby(['Студент']).agg({'sem_start': 'min', 'sem_end': 'max'})

df_kol_let = (df_years['sem_end']).astype('int16') - (df_years['sem_start']).astype('int16')
print(df_kol_let.shape)
(df_kol_let <= 4).sum()

df_kol_let = df_kol_let[df_kol_let > 4]


# df_kol_let = pd.read_csv('TempData.csv')



(1536,)


In [330]:
df_kol_let.to_frame()

df_kol_let.to_csv('TempData.csv', index=True)
df_kol_let = pd.read_csv('TempData.csv')

df_kol_let.drop(df_kol_let.columns[1], axis=1, inplace=True)

df = df.merge(df_kol_let, indicator=True, how='outer').loc[lambda x: x['_merge'] == 'left_only'].drop(columns=['_merge']).drop_duplicates()




In [331]:
df.to_csv('TempData1.csv', index=False)

df.drop_duplicates(inplace=True)

df = df[~(df['Оценка'] == 'неудовлетворительно') | 
        ~(df.duplicated(subset=df.columns.difference(['Оценка']))) ]

df['Оценка'] = df['Оценка'].map({
    'отлично': 5,
    'хорошо': 4,
    'удовлетворительно': 3,
    'зачтено': 2, 
    'незачет': 1 
})

In [332]:
df.drop('Статус', axis=1, inplace=True)
df.drop('Группа', axis=1, inplace=True)
df.drop('Специальность', axis=1, inplace=True)
df.drop('Квалификация', axis=1, inplace=True)

In [334]:
df_max = df.groupby(['Студент', 'Учебный год', 'Дисциплина', 'Семестр']).agg({'Оценка': 'max'})
df_min = df.groupby(['Студент', 'Учебный год', 'Дисциплина', 'Семестр']).agg({'Оценка': 'min'})

df = df_max.merge(df_min, on=['Студент', 'Учебный год', 'Дисциплина', 'Семестр'])

df.sort_values(by=['Студент',"Учебный год", 'Семестр'], inplace=True)

In [338]:
df.head(100)

Оценка_x  \
Студент                                          Учебный год Дисциплина                                         Семестр             
2B41566C72437872494D356F7673772B4731724C4D413D3D 2015/2016   Введение в инженерную деятельность                 1               2   
                                                             Иностранный язык (английский)                      1               2   
                                                             Информатика 1.1                                    1               2   
                                                             История                                            1               4   
                                                             Математика 1.1                                     1               3   
...                                                                                                                           ...   
2B4157422F3966352B2B56416A59384534744C6A39513D3D 2016/2017   Иностранный язык (английский)                      4               4   
                                                             Материаловедение и технология конструкционных м... 4               4   
                                                             Метрология, стандартизация и сертификация 1.1      4               2   
                                                             Механика 2.2                                       4               4   
                                                             Творческий проект                                  4               2   

                                                                                                                         Оценка_y  
Студент                                          Учебный год Дисциплина                                         Семестр            
2B41566C72437872494D356F7673772B4731724C4D413D3D 2015/2016   Введение в инженерную деятельность                 1               2  
                                                             Иностранный язык (английский)                      1               2  
                                                             Информатика 1.1                                    1               2  
                                                             История                                            1               4  
                                                             Математика 1.1                                     1               3  
...                                                                                                                           ...  
2B4157422F3966352B2B56416A59384534744C6A39513D3D 2016/2017   Иностранный язык (английский)                      4               4  
                                                             Материаловедение и технология конструкционных м... 4               4  
                                                             Метрология, стандартизация и сертификация 1.1      4               2  
                                                             Механика 2.2                                       4               2  
                                                             Творческий проект                                  4               2  

[100 rows x 2 columns]